# Forecast Model Builder Deployment

This notebook deploys the Forecast Model Builder files to your Snowflake account. 

The Forecast Model Builder is a tool for efficiently building forecasts.  It includes a collection of notebooks (exploratory data analysis, modeling, and inference) and an orchestration layer for iterating on multiple projects.

# Instructions

__Refer to the [README](https://github.com/Snowflake-Labs/emerging-solutions-toolbox/blob/main/framework-forecast-model-builder/README.md) for detailed instructions.__ 

NOTE: This notebook creates several Snowflake objects. It is recommending to use the default settings in following python cell, if the user has privileges to CREATE DB and WH. Users without those privileges who specify an __existing__ database and schema below, should follw the README instruction option that matches their privilege level. 

In [ ]:
# -----------------------------------------------------------------------------------------
# Solution constants
# -----------------------------------------------------------------------------------------

# Establish the name of the database and warehouse to be used for the solution
# NOTE: User can specify an existing database and warehouse, or can specify a new database and warehouse to be created.
# NOTE: If the following database and/or warehouse do not exist,
#       this notebook will try to create them (assuming user's role has appropriate PRIVILEGES to create objects)
SOLUTION_DB = "FORECAST_MODEL_BUILDER"
DEPLOYMENT_WH = "FORECAST_MODEL_BUILDER_WH"

# Establish the name of the schema that will be used to store the base objects (like staged files, stored procedures, etc)
# NOTE: If SOLUTION_DB specified an already-existing db, then user must specify a schema that ALREADY EXISTS in that db.
#       If SOLUTION_DB specified a new db to be created, then user must specify the name of a new schema that will be created in the db.
SOLUTION_BASE_SCHEMA = "BASE"

PROJECT_NAME = "TEST_PROJECT"

# Git integration
# If None, will attempt to create a git integration
GIT_INTEGRATION = None

# Establish the name of the stage to deploy the notebook templates into
# NOTE: If SOLUTION_DB specified an already-existing db, then user must specify the name of a stage that ALREADY EXISTS in SOLUTION_BASE_SCHEMA.
#       If SOLUTION_DB specified a new db to be created, then user must specify the name of a new stage that will be created in SOLUTION_BASE_SCHEMA.
DEPLOYMENT_STAGE = "DATA_STAGE"

In [ ]:
# Deploys the solution.  Creates a database if it doesn't yet
# exist, and adds supporting schemas and stages, a default warehouse, and
# will add a git repository to automatically load files.
# Will check for needed permissions and will notify if missing.
# *Note* - the role running the notebook will own the objects.

# Import python packages
import os
import zipfile

import pandas as pd
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col

session = get_active_session()

# Solution constant to specify which framework directory to deploy from the Emerging Solutions Toolbox
TOOLBOX_FOLDER_NAME = "framework-forecast-model-builder"

# Permission variables
can_create_db = False
can_create_wh = False
can_create_integration = False

# Deployment variables
database_deployed = False
warehouse_deployed = False
files_deployed = False
zip_deployed = False
git_repository_deployed = False
confirm_message_sent = False


# Checks the permissions of the current role
def check_permissions(session):
    global can_create_db
    global can_create_wh
    global can_create_integration

    # Checks permissions of current role
    current_role_sql = """SELECT CURRENT_ROLE()"""

    current_role = session.sql(current_role_sql).collect()[0][0]

    admin_role_list = ["ACCOUNTADMIN", "SYSADMIN"]

    if current_role not in admin_role_list:
        grants_sql = """SHOW GRANTS ON ACCOUNT"""
        grants_df = session.sql(grants_sql)

        create_db_df = grants_df.filter(
            (col('"privilege"') == "CREATE DATABASE")
            & (col('"grantee_name"') == current_role)
        )

        if create_db_df.count() > 0:
            can_create_db = True

        create_wh_df = grants_df.filter(
            (col('"privilege"') == "CREATE WAREHOUSE")
            & (col('"grantee_name"') == current_role)
        )

        if create_wh_df.count() > 0:
            can_create_wh = True

        create_integration_df = grants_df.filter(
            (col('"privilege"') == "CREATE INTEGRATION")
            & (col('"grantee_name"') == current_role)
        )

        if create_integration_df.count() > 0:
            can_create_integration = True

    else:
        can_create_db = True
        can_create_wh = True
        can_create_integration = True


# Deploys the database and supporting objects
def deploy_database(session):
    global database_deployed

    # Check if database exists
    db_check_sql = f"""SHOW DATABASES LIKE '{SOLUTION_DB}'"""
    db_check_df = session.sql(db_check_sql)

    if db_check_df.count() == 0 and can_create_db:
        try:
            # Create a database for the toolkit
            create_db_sql = f"""CREATE DATABASE IF NOT EXISTS {SOLUTION_DB}
            COMMENT = '{{"origin":"sf_sit", "name":"sit_forecasting", "version":{{"major":1, "minor":0}}, "attributes":{{"component":"deployment"}}}}'"""

            session.sql(create_db_sql).collect()
        except Exception as e:
            print(
                f"⚠️ {SOLUTION_DB} database not available and could not be created.  Please change your role or reach out to an admin.  Error: "
                + str(e)
            )
        finally:
            # Remove the public schema (only during initial deployment)
            remove_public_schema_sql = f"""DROP SCHEMA IF EXISTS {SOLUTION_DB}.PUBLIC"""

            session.sql(remove_public_schema_sql).collect()

            # Create a schema for the toolkit itself (other schemas will be created for each project)
            create_base_schema_sql = f"""CREATE SCHEMA IF NOT EXISTS {SOLUTION_DB}.{SOLUTION_BASE_SCHEMA}
        COMMENT = '{{"origin":"sf_sit", "name":"sit_forecasting", "version":{{"major":1, "minor":0}}, "attributes":{{"component":"deployment"}}}}'"""

            session.sql(create_base_schema_sql).collect()

            # Create a stage for the data
            create_stage_sql = f"""CREATE STAGE IF NOT EXISTS {SOLUTION_DB}.{SOLUTION_BASE_SCHEMA}.{DEPLOYMENT_STAGE}
        DIRECTORY = (ENABLE = TRUE)
        COMMENT = '{{"origin":"sf_sit", "name":"sit_forecasting", "version":{{"major":1, "minor":0}}, "attributes":{{"component":"deployment"}}}}'"""

            session.sql(create_stage_sql).collect()

            database_deployed = True

    elif db_check_df.count() == 0 and not can_create_db:
        print(
            f"⚠️ {SOLUTION_DB} database not available and the current role does not have the CREATE DATBASE permission.  Please change your role or reach out to an admin."
        )

    elif db_check_df.count() == 1:
        database_deployed = True


# Deploys the warehouse
def deploy_warehouse(session):
    global warehouse_deployed

    # Check if warehouse exists
    wh_check_sql = f"""SHOW WAREHOUSES LIKE '{DEPLOYMENT_WH}'"""
    wh_check_df = session.sql(wh_check_sql)

    if wh_check_df.count() == 0 and can_create_wh:
        # Create a warehouse for the toolkit
        create_wh_sql = f"""CREATE WAREHOUSE IF NOT EXISTS {DEPLOYMENT_WH}
    WITH WAREHOUSE_SIZE = 'XSMALL'
    WAREHOUSE_TYPE = 'STANDARD'
    AUTO_SUSPEND = 10
    AUTO_RESUME = TRUE
    INITIALLY_SUSPENDED = TRUE
    COMMENT = '{{"origin":"sf_sit", "name":"sit_forecasting", "version":{{"major":1, "minor":0}}, "attributes":{{"component":"deployment"}}}}'"""

        session.sql(create_wh_sql).collect()

        warehouse_deployed = True

    elif wh_check_df.count() == 0 and not can_create_wh:
        print(
            f"ℹ️ {DEPLOYMENT_WH} warehouse not available, either use your own or rerun with a role with the CREATE WAREHOUSE permission."
        )

    elif wh_check_df.count() == 1:
        warehouse_deployed = True


# Checks if files in the stage IS missing
def check_stage(session):
    global files_deployed

    # Check if files are missing
    files_check_sql = f"""LS @{SOLUTION_DB}.{SOLUTION_BASE_SCHEMA}.{DEPLOYMENT_STAGE}/emerging-solutions-toolbox-main/"""
    if session.sql(files_check_sql).count() > 0:
        files_deployed = True


# Deploys the git repository
def deploy_api_integration(session):
    global GIT_INTEGRATION
    if database_deployed and not files_deployed:
        check_git_repository(session)
        check_for_zip(session)

        if git_repository_deployed:
            add_files_from_git(session)

        elif zip_deployed:
            add_files_from_zip(session)

        else:
            print(
                f"ℹ️ If you do not want to use git, you can manually upload the zip file from the repository to {SOLUTION_DB}.{SOLUTION_BASE_SCHEMA}.{DEPLOYMENT_STAGE} and rerun this cell."
            )

            if GIT_INTEGRATION:
                deploy_git(session)
            else:
                if can_create_integration:
                    api_integration_name = "SNOWFLAKE_LABS_GIT_API_INTEGRATION"  # Set API integration name here
                    print(f"Creating API Integration: {api_integration_name}")
                    api_integration_create_sql = f"""CREATE OR REPLACE API INTEGRATION {api_integration_name}
        API_PROVIDER = git_https_api
        API_ALLOWED_PREFIXES = ('https://github.com/Snowflake-Labs/')
        ENABLED = TRUE;"""
                    session.sql(api_integration_create_sql).collect()
                    GIT_INTEGRATION = api_integration_name
                    deploy_git(session)

                else:
                    print(
                        "⚠️ There are no API integrations and the current role does not have permission to create one or contact your admin or manually stage the files and rerun this cell."
                    )
                
    elif database_deployed and files_deployed:
        # Check if git is added for status
        check_git_repository(session)
        check_for_zip(session)


# Adds a git repository to the database
def deploy_git(session):
    global files_deployed
    global confirm_message_sent
    global git_repository_deployed

    if GIT_INTEGRATION:
        print(f"Using API Integration: {GIT_INTEGRATION}")
        
        try:
            repo_sql = f"""CREATE GIT REPOSITORY IF NOT EXISTS {SOLUTION_DB}.{SOLUTION_BASE_SCHEMA}.EMERGING_SOLUTIONS_TOOLBOX
API_INTEGRATION = "{GIT_INTEGRATION}"
ORIGIN = 'https://github.com/Snowflake-Labs/emerging-solutions-toolbox.git'"""

            session.sql(repo_sql).collect()
            add_files_from_git(session)
            git_repository_deployed = True

        except Exception as e:
            print(
                "⚠️ Could not create repository.  Try another API Integration or contact an admin.\n  Error: "
                + str(e)
            )
    else:
        print("⚠️ No git API integrations available. Please create one or contact your admin.")

    confirmation_message()


# Writes a confirmation message if not already written
def confirmation_message():
    global confirm_message_sent

    if database_deployed and files_deployed:
        if not confirm_message_sent:
            status_df = pd.DataFrame(
                [
                    ["Database Deployed", database_deployed],
                    ["Warehouse Deployed", warehouse_deployed],
                    [
                        "Git Repository or Zip Deployed",
                        git_repository_deployed or zip_deployed,
                    ],
                    ["Files Deployed", files_deployed],
                ],
                columns=["Step", "Complete"],
            )

            display(status_df)

            print(
                "✓ Solution from " + str(TOOLBOX_FOLDER_NAME) + " fully deployed!"
            )
            confirm_message_sent = True


# Checks if the git repository has been added
def check_git_repository(session):
    global git_repository_deployed

    # Check for git repository
    git_repository_sql = f"""SHOW GIT REPOSITORIES LIKE 'EMERGING_SOLUTIONS_TOOLBOX' IN DATABASE {SOLUTION_DB}"""
    git_repository_df = session.sql(git_repository_sql)

    if git_repository_df.count() > 0:
        git_repository_deployed = True


# Checks if the zip file has been uploaded
def check_for_zip(session):
    global zip_deployed

    # Check for zip file
    zip_check_sql = f"""LS @{SOLUTION_DB}.{SOLUTION_BASE_SCHEMA}.{DEPLOYMENT_STAGE} PATTERN='.*emerging-solutions-toolbox-main.zip'"""
    zip_check_df = session.sql(zip_check_sql)

    if zip_check_df.count() > 0:
        zip_deployed = True


# Adds the notebooks and python files to the stage from the git repository
def add_files_from_git(session):
    global files_deployed

    if not files_deployed:
        copy_notebooks_sql = f"""COPY FILES
INTO @{SOLUTION_DB}.{SOLUTION_BASE_SCHEMA}.{DEPLOYMENT_STAGE}/emerging-solutions-toolbox-main/
FROM @{SOLUTION_DB}.{SOLUTION_BASE_SCHEMA}.EMERGING_SOLUTIONS_TOOLBOX/branches/main/"""

        session.sql(copy_notebooks_sql).collect()
        files_deployed = True


# Adds the notebooks and python files to the stage from a zip file
def add_files_from_zip(session):
    global files_deployed

    if not files_deployed:
        f = session.file.get_stream(
            f"@{SOLUTION_DB}.{SOLUTION_BASE_SCHEMA}.{DEPLOYMENT_STAGE}/emerging-solutions-toolbox-main.zip"
        )
        with zipfile.ZipFile(f, "r") as git_zip:
            git_zip.extractall()

        dirname = [f for f in os.listdir('.') if f.startswith("emerging-solutions-toolbox")][0]
        os.rename(dirname, "emerging-solutions-toolbox-main")

        path_list = [
            os.path.join(dirpath, f)
            for (dirpath, dirnames, filenames) in os.walk(
                "emerging-solutions-toolbox-main"
            )
            for f in filenames
        ]

        for path in path_list:
            directory, file_name = os.path.split(path)

            put_result = session.file.put(
                path,
                f"@{SOLUTION_DB}.{SOLUTION_BASE_SCHEMA}.{DEPLOYMENT_STAGE}/{directory}",
                auto_compress=False,
            )

        files_deployed = True


# Deploys code specific for the solution
def deploy_solution_specific_code(session):
    if files_deployed:
        # Deploy sample data
        sample_data_deployed = False

        sample_data_check_sql = f"""SHOW TABLES LIKE 'DAILY_PARTITIONED_SAMPLE_DATA' IN SCHEMA {SOLUTION_DB}.{SOLUTION_BASE_SCHEMA}"""
        if session.sql(sample_data_check_sql).count() > 0:
            sample_data_deployed = True

        if not sample_data_deployed:
            create_file_format_sql = f"""CREATE OR REPLACE FILE FORMAT {SOLUTION_DB}.{SOLUTION_BASE_SCHEMA}.CSV_FORMAT 
        TYPE = CSV
        FIELD_DELIMITER = ','
        PARSE_HEADER = TRUE
        COMMENT = '{{"origin":"sf_sit", "name":"sit_forecasting", "version":{{"major":1, "minor":0}}, "attributes":{{"component":"deployment"}}}}'"""

            session.sql(create_file_format_sql).collect()

            create_table_sql = f"""CREATE OR REPLACE TABLE {SOLUTION_DB}.{SOLUTION_BASE_SCHEMA}.DAILY_PARTITIONED_SAMPLE_DATA USING TEMPLATE (
    SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*)) 
     FROM TABLE (INFER_SCHEMA(
     LOCATION=>'@{SOLUTION_DB}.{SOLUTION_BASE_SCHEMA}.{DEPLOYMENT_STAGE}/emerging-solutions-toolbox-main/{TOOLBOX_FOLDER_NAME}/sample_data/daily_partitioned_sample_data.csv',
     FILE_FORMAT=>'{SOLUTION_DB}.{SOLUTION_BASE_SCHEMA}.CSV_FORMAT')))"""
            session.sql(create_table_sql).collect()

            tag_table_sql = f"""ALTER TABLE {SOLUTION_DB}.{SOLUTION_BASE_SCHEMA}.DAILY_PARTITIONED_SAMPLE_DATA SET COMMENT = '{{"origin":"sf_sit", "name":"sit_forecasting", "version":{{"major":1, "minor":0}}, "attributes":{{"component":"deployment"}}}}'"""

            session.sql(tag_table_sql).collect()

            load_table_sql = f"""COPY INTO {SOLUTION_DB}.{SOLUTION_BASE_SCHEMA}.DAILY_PARTITIONED_SAMPLE_DATA
    from '@{SOLUTION_DB}.{SOLUTION_BASE_SCHEMA}.{DEPLOYMENT_STAGE}/emerging-solutions-toolbox-main/{TOOLBOX_FOLDER_NAME}/sample_data/daily_partitioned_sample_data.csv'
    FILE_FORMAT = (SKIP_HEADER = 1)"""

            session.sql(load_table_sql).collect()

    create_schema_sql = f"""CREATE SCHEMA IF NOT EXISTS {SOLUTION_DB}.{PROJECT_NAME}
    COMMENT = '{{{{"origin":"sf_sit", "name":"sit_forecasting", "version":{{{{"major":1, "minor":0}}}}, "attributes":{{{{"component":"deployment"}}}}}}}}'"""

    session.sql(create_schema_sql).collect()

    return f"""Project schema created. Use Git integration to manage notebooks in Workspaces."""


check_permissions(session)
deploy_database(session)
deploy_warehouse(session)
check_stage(session)
deploy_api_integration(session)
deploy_solution_specific_code(session)
confirmation_message()

**Next Steps**

Once you've created your project, go to your Notebooks and start with <your project name>__EDA